In [ ]:
from optimization.evolutionary.geneticalgorithms import GeneticSystem, GeneticEncoder, BitStringBaseTypes, SplitCrossOver, PointMutator, ProportionateSelector

class TestEncoder(GeneticEncoder):
    def encode(self, solution):
        return solution
    def decode(self, solution):
        return solution
    def evaluate_fitness(self, chromosome):
        return sum(1 if ((int(x) % 2) == (i % 2)) else 0 for i, x in enumerate(chromosome))

encoder = TestEncoder(20, False, BitStringBaseTypes.bin)
selector = ProportionateSelector()
recombiner = SplitCrossOver()
mutator = PointMutator(1)
system = GeneticSystem(encoder, selector, recombiner, mutator)

result = system.run(5, 50, 1.5, 0.75, mutation_factor_final=0.0)
print(result)



In [ ]:
from auxiliary.moreitertools import chunk
from optimization.evolutionary.geneticalgorithms import GeneticSystem, GeneticEncoder, BitStringBaseTypes, SplitCrossOver, PointMutator, ProportionateSelector

import matplotlib.pyplot as plt

from control.pid import PIDController
from control.systems import InvertedPendulumSystem
from control.controlutils import simulate_control

pend_system = InvertedPendulumSystem()
controller = PIDController(0.0, 0.0, 0.0, initial_error=pend_system.get_control_input() - pend_system.get_setpoint())

ticks: int = 100
delta_time: float = 0.1

def control_evaluator(vec):
    controller.reset()
    controller.set_gains(*vec)
    pend_system.reset()
    return simulate_control(pend_system, controller, ticks, delta_time)

class TestEncoder(GeneticEncoder):
    def encode(self, solution):
        return solution
    def decode(self, solution):
        return solution
    def evaluate_fitness(self, chromosome):
        return -control_evaluator([int(c, base=2) / 100 for c in chunk(chromosome, 10)])

encoder = TestEncoder(30, False, BitStringBaseTypes.bin)
selector = ProportionateSelector()
recombiner = SplitCrossOver()
mutator = PointMutator(1)
system = GeneticSystem(encoder, selector, recombiner, mutator)

result = system.run(500, 500, 1.5, 0.75, mutation_factor_final=0.0, max_generations=500)
print(result)
print([int(c, base=2) / 100 for c in chunk(result.best_individual, 10)])